## Creating Context for the LLM

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import os 
from dotenv import load_dotenv


In [ ]:
load_dotenv()
groq_api_key=os.getenv('GROQ_API_KEY')

In [ ]:
loader = PyPDFLoader("Ip102_Impact.pdf")
pages = loader.load()

In [ ]:
rows = []
for page in pages:
    for line in page.page_content.split("\n"):
        parts = line.strip().split()
        if len(parts) >= 5 and parts[0].lower() != "pest":
            rows.append(line.strip())

In [ ]:
docs = [Document(page_content=row) for row in rows]
print("Sample document:", docs[32])

In [ ]:
embeddings = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
vector_db = FAISS.from_documents(docs,embeddings)
vector_db
vector_db.save_local('faiss_index')

In [ ]:
rst=vector_db.similarity_search('Corn Leaf Miner Corn Medium')
rst[1].page_content

In [ ]:
retriever= vector_db.as_retriever()

In [ ]:
retriever.invoke("mango")

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain_groq import ChatGroq
from langchain_core.documents import Document

In [ ]:
llm = ChatGroq(api_key=groq_api_key, model='Gemma2-9b-It')

In [ ]:
prompt= ChatPromptTemplate.from_messages([
    ("system","You are an impact analyzer agent for a smart farming system," 
     "use the context below to provide accurate and precise impact prediction based on the user input."
     "provide the user with Risk Level, Avg Damage, Predicted Yield Loss."),
     ("system","<context>\n{context}\n</context>"),
     ("human", "Question:{input} and {croptype}")
])

In [ ]:
chain = create_stuff_documents_chain(llm,prompt)

In [ ]:
chain.invoke({
      "input": "Green Rice Leafhopper ",
      "croptype":"rice",
      "context":[Document(page_content="'Green Rice LeafhopperRice Medium 12 8\nRice Gall Midge Rice High 18 12\nRice Stem Borer Rice High 22 17\nRice Brown LeafhopperRice Medium 14 10\nRice Green LeafhopperRice Medium 12 8')")]

})

In [ ]:
ret_chain = create_retrieval_chain(retriever,chain)
ret_chain

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
result = ret_chain.invoke({    "input": "Green Rice Leafhopper", "croptype": "Rice" })
result['answer']


In [ ]:
parser =StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableLambda
chain = ret_chain | RunnableLambda(lambda x: x["answer"]) | parser

In [ ]:
parsed = chain.invoke({"input": "Green Rice Leafhopper","croptype":" Rice"})
print(parsed)

## Save the models and artifacts